# Hugging Face Qwen Structured Output Example

This example demonstrates using `HuggingFaceBackend` with a generic LLM (Qwen) to generate Structured Outputs (Pydantic objects) by parsing the response.

In [1]:
from pydantic import BaseModel, Field

from grounded_ai import Evaluator


# 1. Define your Custom Schema
class AIAnalysis(BaseModel):
    summary: str = Field(description="A concise summary of the input text.")
    sentiment: str = Field(
        description="The sentiment, e.g., Positive, Negative, Neutral."
    )
    keywords: list[str] = Field(description="Key topics found in the text.")


# 2. Initialize Evaluator
# Uses your new HuggingFaceBackend logic (AutoModel + LogitsProcessor)
evaluator = Evaluator(
    "hf/Qwen/Qwen2.5-1.5B-Instruct",  # Corrected from user typo Qwen3-1.7B to valid model for demo
    task="text-generation",
    device="cuda",  # or "mps", "cpu"
    system_prompt="You are an intelligent data analyst. Extract insights as structured JSON.",
)

In [2]:
# 3. Evaluate
from grounded_ai.schemas import EvaluationInput

long_text_input = """
I've been on hold for 45 MINUTES and nobody can tell me why I'm being charged TWICE for this garbage platform!
Your so-called "QuantumStream" doesn't even WORK on my phone - every time I try to manage my subscription, the whole app crashes!
I signed up specifically because you advertised a Family Plan for my kids, and guess what? IT DOESN'T EXIST!
You people are running a SCAM! I want my money back RIGHT NOW!
And don't even get me started on trying to find the damn settings - I have to click through three different menus just to change the volume!
My neighbor told me everyone's ditching this platform because it's completely unusable, and now I see why.
I'm calling my bank to dispute these charges if I don't get a refund in the next 24 hours. This is ridiculous!
"""

result = evaluator.backend.evaluate(
    EvaluationInput(response=long_text_input),
    output_schema=AIAnalysis,
    temperature=0.1,
    max_new_tokens=1024,
)
result

AIAnalysis(summary="Customer complains about QuantumStream's poor service, including app crashes, double charges, non-existent Family Plan, and usability issues, demanding a refund and dispute resolution.", sentiment='Negative', keywords=['QuantumStream', 'app crashes', 'double charges', 'Family Plan', 'scam', 'unusable', 'dispute charges', 'settings difficulty'])